In [1]:
from preprocessing import *
from sklearn.model_selection import KFold
import argparse
from model import *
from test import *
import torch.optim as optim
from torch.utils.data import Dataset
import pandas as pd

from MatrixVectorizer import *
import networkx as nx
from typing import Union

import random
import pickle


In [2]:
# Set a fixed random seed for reproducibility across multiple libraries
random_seed = 42
random.seed(random_seed)
np.random.seed(random_seed)
torch.manual_seed(random_seed)

device = torch.device("cpu")

In [3]:
# load csvs as numpy
lr_data_path = '../data/lr_train.csv'
hr_data_path = '../data/hr_train.csv'

lr_train_data = pd.read_csv(lr_data_path, delimiter=',').to_numpy()
hr_train_data = pd.read_csv(hr_data_path, delimiter=',').to_numpy()
lr_train_data[lr_train_data < 0] = 0
np.nan_to_num(lr_train_data, copy=False)

hr_train_data[hr_train_data < 0] = 0
np.nan_to_num(hr_train_data, copy=False)

# map the anti-vectorize function to each row of the lr_train_data
lr_train_data_vectorized = torch.tensor([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data],
                                        dtype=torch.float32)
hr_train_data_vectorized = torch.tensor([MatrixVectorizer.anti_vectorize(row, 268) for row in hr_train_data],
                                        dtype=torch.float32)


/tmp/ipykernel_19726/3337449821.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ../torch/csrc/utils/tensor_new.cpp:204.)
  lr_train_data_vectorized = torch.tensor([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_train_data],


In [4]:
class NoisyDataset(Dataset):
    def __init__(self, lr_data, hr_data, noise_level=0.01):
        """
        lr_data: Low resolution data (torch.tensor)
        hr_data: High resolution data (torch.tensor)
        noise_level: Standard deviation of Gaussian noise to be added
        """
        self.lr_data = lr_data
        self.hr_data = hr_data
        self.noise_level = noise_level

    def __len__(self):
        return len(self.lr_data)

    def __getitem__(self, idx):
        lr_sample = self.lr_data[idx]
        hr_sample = self.hr_data[idx]

        # Adding Gaussian noise
        noise = torch.randn(lr_sample.size()) * self.noise_level
        noisy_lr_sample = lr_sample + noise

        # Clipping to ensure values are between 0 and 1
        noisy_lr_sample = torch.clamp(noisy_lr_sample, 0, 1)

        return noisy_lr_sample, hr_sample

train_data = NoisyDataset(lr_train_data_vectorized, hr_train_data_vectorized, noise_level=0.5)
train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, shuffle=True)

In [5]:
num_splt = 3
epochs = 200
lr = 0.00005 # try [0.0001, 0.0005, 0.00001, 0.00005]
lmbda = 17 # should be around 15-20
lamdba_topo = 0.0005 # should be around 0.0001-0.001
lr_dim = 160
hr_dim = 320
hidden_dim = 320 # try smaller and larger - [160-512]
padding = 26
dropout = 0.2 # try [0., 0.1, 0.2, 0.3]


args = argparse.Namespace()
args.epochs = epochs
args.lr = lr
args.lmbda = lmbda
args.lamdba_topo = lamdba_topo
args.lr_dim = lr_dim
args.hr_dim = hr_dim
args.hidden_dim = hidden_dim
args.padding = padding
args.p = dropout


In [6]:
cv = KFold(n_splits=3, random_state=42, shuffle=True)

In [7]:
ks = [0.9, 0.7, 0.6, 0.5]
model = GSRNet(ks, args)

In [8]:
criterion = nn.L1Loss()

In [9]:
def generate_submission_csv(model, data_path='../data/lr_test.csv', filename='submission.csv'):
    lr_test_data = pd.read_csv(data_path, delimiter=',').to_numpy()
    lr_test_data[lr_test_data < 0] = 0
    np.nan_to_num(lr_test_data, copy=False)
    lr_test_data_vectorized = np.array([MatrixVectorizer.anti_vectorize(row, 160) for row in lr_test_data])
    model.eval()
    preds = []
    for lr in lr_test_data_vectorized:      
        lr = torch.from_numpy(lr).type(torch.FloatTensor)
        
        model_outputs, _, _, _ = model(lr)
        model_outputs  = unpad(model_outputs, args.padding)
        preds.append(MatrixVectorizer.vectorize(model_outputs.detach().numpy()))

    r = np.hstack(preds)
    meltedDF = r.flatten()
    n = meltedDF.shape[0]
    df = pd.DataFrame({'ID': np.arange(1, n+1),
                    'Predicted': meltedDF})
    df.to_csv(filename, index=False)

In [10]:
def train(model, train_data_loader, optimizer, args): 
  
  all_epochs_loss = []
  no_epochs = args.epochs

  for epoch in range(no_epochs):
    epoch_loss = []
    epoch_error = []
    epoch_topo = []

    model.train()
    for lr, hr in train_data_loader: 
      lr.to(device)   
      hr.to(device)  
      lr = lr.reshape(160, 160)
      hr = hr.reshape(268, 268)

      model_outputs,net_outs,start_gcn_outs,layer_outs = model(lr)
      model_outputs  = unpad(model_outputs, args.padding)

      padded_hr = pad_HR_adj(hr,args.padding)
      _, U_hr = torch.linalg.eigh(padded_hr, UPLO='U')

      loss = args.lmbda * criterion(net_outs, start_gcn_outs) + criterion(model.layer.weights,U_hr) + criterion(model_outputs, hr) 
      topo = compute_topological_MAE_loss(hr, model_outputs)
      
      loss += args.lamdba_topo * topo

      error = criterion(model_outputs, hr)
      
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      epoch_loss.append(loss.item())
      epoch_error.append(error.item())
      epoch_topo.append(topo.item())
      
  
    model.eval()
    if (epoch + 1) % 100 == 0:
      filename = f'{epoch+1}epoch-model.sav'
      pickle.dump(model, open(filename, 'wb'))
      generate_submission_csv(model, filename=f'{epoch+1}epoch-model.csv')
    print("Epoch: ",epoch+1, "Loss: ", np.mean(epoch_loss), "Error: ", np.mean(epoch_error),
          "Topo: ", np.mean(epoch_topo))
    all_epochs_loss.append(np.mean(epoch_loss))

# Final Model & Kaggle Submission

In [11]:
#final train
final_model = GSRNet(ks, args) #pickle.load(open('300epoch-model.sav', 'rb'))#
optimizer = optim.Adam(final_model.parameters(), lr=args.lr)

final_model.to(device)

train(final_model, train_data_loader, optimizer, args)

Epoch:  1 Loss:  0.44269737547743104 Error:  0.2096005307878563 Topo:  38.09577772860042
Epoch:  2 Loss:  0.2999330361089307 Error:  0.18171630969304525 Topo:  21.267710702861855
Epoch:  3 Loss:  0.2923052475124062 Error:  0.17833045160699035 Topo:  19.844695433885036
Epoch:  4 Loss:  0.28625621445878535 Error:  0.1764510716506821 Topo:  19.436927795410156
Epoch:  5 Loss:  0.2806439662943343 Error:  0.1753702840048396 Topo:  19.24852789804607
Epoch:  6 Loss:  0.27518474199100884 Error:  0.1741512225059692 Topo:  19.01814871348307
Epoch:  7 Loss:  0.26976441641053756 Error:  0.1732635708626159 Topo:  19.16435644869319
Epoch:  8 Loss:  0.26494179041442756 Error:  0.17233109795404766 Topo:  18.988653708360864
Epoch:  9 Loss:  0.2607347944717921 Error:  0.17143990277887106 Topo:  19.010916601398034
Epoch:  10 Loss:  0.25620305011729283 Error:  0.1703671977191628 Topo:  19.08328916926584
Epoch:  11 Loss:  0.25230781992752394 Error:  0.1693758663838495 Topo:  18.878896810337455
Epoch:  12 Lo

In [12]:
import pickle
filename = 'final-model.sav'
pickle.dump(final_model, open(filename, 'wb'))

In [13]:
generate_submission_csv(final_model)